In [1]:
import tensorflow as _tf
from syft.tensor import tensorflow as tf

# Tensorflow Eager Wishlist:

- build something which feels like you're subclassing tensorflow even if you're not
- GPU support
- autograd support

In [3]:
# class FixedPrecisionTensor(AbstractTorchTensor):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         self.base = 10
#         self.precision_fractional = 3

#         self.data = self.data * self.scaling_factor

#     @property
#     def scaling_factor(self):
#         return self.base ** self.precision_fractional


In [49]:
tf.AbstractTensor.on = on
tf.Variable.on = on
tf.ResourceVariable.on = on

In [52]:
w = tf.Variable([[100.0]])
w = w.on(tf.MinusIsMultiplyTensor).on(tf.PlusIsMinusTensor)

In [53]:

# w = ResourceVariable([[100.0]])
with _tf.GradientTape() as tape:
    loss = w + w

grad = tape.gradient(loss, w)

In [54]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [55]:
loss

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[10000.]], dtype=float32)>

In [ ]:

@tf.custom_gradient
def _add(self, other):

    def grad(dy):
        return dy, dy
    
    result = tf.subtract(self, other)
    
    return result, grad

def add(self, other):
    return _add(self, other)

In [7]:

w = tf.Variable([[100.0]])
# w = ResourceVariable([[100.0]])
with tf.GradientTape() as tape:
    loss = add(w,w)

grad = tape.gradient(loss, w)

In [6]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [ ]:
# Build the model
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

In [88]:
for images,labels in dataset.take(1):
  print("Logits: ", mnist_model(images[0:1]).numpy())

Logits:  [[-0.04930329  0.00467802 -0.00234521 -0.03856605  0.006839   -0.03424904
   0.05314591 -0.01709108  0.00721264 -0.01741508]]


In [93]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []

In [94]:
def train_step(images, labels):
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)
    
    # Add asserts to check the shape of the output.
    tf.debugging.assert_equal(logits.shape, (32, 10))
    
    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

In [95]:
def train(epochs):
  for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(dataset):
      train_step(images, labels)
    print ('Epoch {} finished'.format(epoch))

In [ ]:
train(epochs = 3)

Epoch 0 finished


In [34]:
@tf.custom_gradient
def log1pexp2(self):

    result = simplelog2pexp(self)

    def grad(dy):
        return dy * (1 - 1 / (1 + tf.exp(self)))

    return result, grad

def outer_log1pexp(self):
    return log1pexp2(self)

def simplelog2pexp(self):
    e = tf.exp(self)
    result = tf.math.log(1 + e)
    return result
    

# def log1pexp(x):
#   return tf.math.log(1 + tf.exp(x))

tf.Tensor.log1pexp2 = simplelog2pexp
tf.Variable.log1pexp2 = outer_log1pexp

w = tf.Variable([[100.0]])
with tf.GradientTape() as tape:
    ResourceVariable =type(w)
    
ResourceVariable.log1pexp2 = outer_log1pexp

In [35]:
a = tf.constant([[1, 2],
                 [3, 4.]])

In [36]:
a.log1pexp2()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.3132616, 2.126928 ],
       [3.0485873, 4.01815  ]], dtype=float32)>

In [37]:
w = tf.Variable([[100.0]])
# w = ResourceVariable([[100.0]])
with tf.GradientTape() as tape:
    loss = w.log1pexp2()

grad = tape.gradient(loss, w)

In [33]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>